- SUCH 2022
- WITH TRANSPORTATION AT ALL CPs

In [1]:
import pandas as pd
import numpy as np
import requests
import ast
import json
import tqdm
import geopandas as gpd
import matplotlib.pyplot as plt
import os
import datetime
import tqdm

from dotenv import load_dotenv

from utils.googlemaps.api import gm_distance_api, chunks
from utils.reporting.reports import getRouteReport
from utils.gtsp.GTSPutils import createDistanceMatrix, createDummyNode, createClusters, createGTSPfile, loadOptimalTour, reOrderTour, evaluateTour
from utils.gpsconversion.conversionapi import convertWSG84toMN95

# LOAD ENV VARIABLES

In [2]:
load_dotenv()

DATA_PATH = os.environ['DATA_PATH']

# IMPORT COORDS

In [3]:
data_version = '05'

In [4]:
with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'mandatory_checkpoints.json'), 'r') as file:
    mandatory_checkpoints = json.load(file)
    
mandatory_checkpoints = {k: ast.literal_eval(v) for k, v in mandatory_checkpoints.items()}

#with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'target_coords.json'), 'r') as file:
#    target_coords = json.load(file)
#    
#target_coords= {k: ast.literal_eval(v) for k, v in target_coords.items()}

In [5]:
with open(os.path.join(DATA_PATH, 'DIST_MATRIX', f'checkpoints_{data_version}.json'), 'r') as file:
    target_coords = json.load(file)
    
target_coords= {k: tuple(v) for k, v in target_coords.items()}

In [6]:
checkpoints = mandatory_checkpoints.copy()
checkpoints.update(target_coords)

In [7]:
checkpoints

{'st_imier_haut': (47.158679, 6.992699),
 'st_imier_bas': (47.15127, 6.99283),
 'lenzerheide_haut': (46.72093, 9.530324),
 'lenzerheide_bas_lift': (46.724895, 9.547017),
 'lenzerheide_bas_sporz': (46.715323, 9.540237),
 'beckenried': (46.967084, 8.475212),
 'gersau': (46.990807, 8.525738),
 'goschenen': (46.665698, 8.588439),
 'sanetsch': (46.363102, 7.295129),
 'versoix': (46.295182, 6.167042),
 'mies': (46.296179, 6.168144),
 'st_maurice': (46.223444, 7.003184),
 'diablerets': (46.361971, 7.228706),
 'rougemont': (46.486861, 7.228648),
 'chatel_st_denis': (46.512836, 6.911657),
 'vaumarcus': (46.870955, 6.754053),
 'chaux_de_fond': (47.122309, 6.885278),
 'chasseral': (47.125755, 7.038184),
 'delemont': (47.375549, 7.3615),
 'liesberg': (47.397281, 7.400262),
 'liesberg_2': (47.402422, 7.443518),
 'basel': (47.535692, 7.609889),
 'kaiseraugst': (47.536199, 7.721017),
 'kaiserstuhl': (47.564849, 8.420254),
 'aadorf': (47.494296, 8.894507),
 'wil': (47.466492, 9.029698),
 'herisau': (4

# IMPORT CP OPENING TIMES

In [8]:
with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'cp_opening_times.json'), 'w') as outfile:

    json.dump(cp_opening_times, outfile, indent=4, default=str)

NameError: name 'cp_opening_times' is not defined

In [9]:
with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'cp_opening_times.json'), 'r') as file:
    cp_opening_times = json.load(file)
    
#cp_opening_times = {k: ast.literal_eval(v) for k, v in cp_opening_times.items()}

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
cp_opening_times = {'st_imier_bas': [datetime.datetime.strptime('7/9/2022 06:35', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('7/9/2022 23:10', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('8/9/2022 06:35', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('8/9/2022 23:10', '%d/%m/%Y %H:%M')],
                    'st_imier_haut': [datetime.datetime.strptime('7/9/2022 06:35', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('7/9/2022 23:10', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('8/9/2022 06:35', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('8/9/2022 23:10', '%d/%m/%Y %H:%M')], 
                    'lenzerheide_haut': [datetime.datetime.strptime('7/9/2022 09:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('7/9/2022 17:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('8/9/2022 09:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('8/9/2022 17:00', '%d/%m/%Y %H:%M')],
                    'lenzerheide_bas_lift': [datetime.datetime.strptime('7/9/2022 09:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('7/9/2022 17:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('8/9/2022 09:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('8/9/2022 17:00', '%d/%m/%Y %H:%M')],
                    'lenzerheide_bas_sporz': [datetime.datetime.strptime('7/9/2022 09:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('7/9/2022 17:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('8/9/2022 09:00', '%d/%m/%Y %H:%M'),
                                    datetime.datetime.strptime('8/9/2022 17:00', '%d/%m/%Y %H:%M')],
                    'beckenried': [datetime.datetime.strptime('7/9/2022 07:40', '%d/%m/%Y %H:%M'),
                                   datetime.datetime.strptime('7/9/2022 18:40', '%d/%m/%Y %H:%M'),
                                   datetime.datetime.strptime('8/9/2022 07:40', '%d/%m/%Y %H:%M'),
                                   datetime.datetime.strptime('8/9/2022 18:40', '%d/%m/%Y %H:%M')],
                    'gersau': [datetime.datetime.strptime('7/9/2022 07:00', '%d/%m/%Y %H:%M'),
                               datetime.datetime.strptime('7/9/2022 18:00', '%d/%m/%Y %H:%M'),
                               datetime.datetime.strptime('8/9/2022 07:00', '%d/%m/%Y %H:%M'),
                               datetime.datetime.strptime('8/9/2022 18:00', '%d/%m/%Y %H:%M')],
                    'goschenen': [datetime.datetime.strptime('7/9/2022 06:51', '%d/%m/%Y %H:%M'),
                                  datetime.datetime.strptime('7/9/2022 21:51', '%d/%m/%Y %H:%M'),
                                  datetime.datetime.strptime('8/9/2022 06:51', '%d/%m/%Y %H:%M'),
                                  datetime.datetime.strptime('8/9/2022 21:51', '%d/%m/%Y %H:%M')],
                    'sanetsch': [datetime.datetime.strptime('7/9/2022 08:30', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('7/9/2022 17:00', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('8/9/2022 08:30', '%d/%m/%Y %H:%M'),
                                 datetime.datetime.strptime('8/9/2022 17:00', '%d/%m/%Y %H:%M')]}

# LOAD CLUSTER MAPPING

In [ ]:
with open(os.path.join(DATA_PATH, 'CHECKPOINTS', 'cluster_mapping.json'), 'r') as file:
    cluster_mapping = json.load(file)

# RUN CELL WITH DIFFERENT START POINTS TO GENERATE REPORTS

In [11]:
reports, reports_r = evaluateTour(start='versoix',
                                  end='bern',
                                  use_transport=True,
                                  optimize='time',
                                  data_version=data_version,
                                  save_plot=False)

FileNotFoundError: [Errno 2] No such file or directory: 'julia'

In [ ]:
for start in tqdm.tqdm(list(checkpoints.keys())):
    
    reports, reports_r = evaluateTour(start=start,
                                      end='bern',
                                      use_transport=True,
                                      optimize='time',
                                      data_version=data_version,
                                      save_plot=False)

In [ ]:
reports = []
reports_r = []
tour_costs = []

optimize = 'time'

use_transport = True
save_plot = False

end = 'bern'



# LOAD CANTONAL BORDER DATA
###########################################################################

df_cantons = gpd.read_file(os.path.join(DATA_PATH, 'SHP_FILES', 'swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp'))
cantons = df_cantons['NAME'].unique()

# DEFINE CP COORDS IN MN95 FORMAT
###########################################################################

cp_coord_x = [2588950.069, 2677691.166, 2680954.976, 2688009.628, 2566249.259, 2759952.968, 2600422.411]
cp_coord_y = [1134641.528, 1202811.284, 1205098.173, 1168980.565, 1223196.312, 1176579.977, 1199574.686]


for start in tqdm.tqdm(list(checkpoints.keys())):
    
    # LOAD GM API DATA AND CREATE DISTANCE MATRIX
###########################################################################

    if optimize == 'time':

        with open(os.path.join(DATA_PATH, 'DIST_MATRIX', f'api_data_time_{data_version}.json'), 'r') as file:
            api_data = json.load(file)

        df = createDistanceMatrix(api_data, use_transport=use_transport, optimize=optimize)

        with open(os.path.join(DATA_PATH, 'DIST_MATRIX', f'api_data_dist_{data_version}.json'), 'r') as file:
            api_data_distance = json.load(file)

        df_distance = createDistanceMatrix(api_data_distance, use_transport=use_transport, optimize='distance')

    elif optimize == 'distance':

        with open(os.path.join(DATA_PATH, 'DIST_MATRIX', f'api_data_dist_{data_version}.json'), 'r') as file:
            api_data = json.load(file)

        df = createDistanceMatrix(api_data, use_transport=use_transport, optimize=optimize)

        df_distance = df.copy()
    
    # CREATE DUMMY NODE
    ###########################################################################
    
    df = createDummyNode(df, start, end, checkpoints)
    
    # ADD CLUSTER
    ###########################################################################
    
    such_2022, clusters = createClusters(checkpoints, cluster_mapping, data_version)
    
    # CREATE GTSP FILE
    ###########################################################################
    
    createGTSPfile(df, clusters)
    
    # RUN JULIA SOLVER
    ###########################################################################

    !/Applications/Julia-1.7.app/Contents/Resources/julia/bin/julia ../scripts/gtsp_solver.jl
    
    # LOAD OPTIMAL TOUR
    ###########################################################################
    
    tour, tour_cost = loadOptimalTour('tour_test_2022.txt')
    
    tour_costs.append(tour_cost)

    # RE-ORDER OPTIMAL TOUR
    ###########################################################################

    tour_coord_x, tour_coord_y, df_tour = reOrderTour(such_2022, tour, cluster_mapping)

    # PLOT OPTIMAL TOUR
    ###########################################################################
    
    fig, ax = plt.subplots(figsize=(25,13))

    for canton in cantons:
        x_c, y_c = df_cantons[df_cantons['NAME']==canton]['geometry'].iloc[0].exterior.coords.xy
        x_c = list(x_c)[:-1]
        y_c = list(y_c)[:-1]
        ax.plot(x_c, y_c)

    ax.plot(cp_coord_x, cp_coord_y, 'r*', markersize=15)

    ax.plot(tour_coord_x, tour_coord_y, '-o', markersize=15)

    for i, (x, y) in enumerate(zip(tour_coord_x, tour_coord_y)):

        ax.text(x, y, i, fontsize='large')

    ax.text(2830000, 1300000, f'cost: {tour_cost}', fontsize='large')
    ax.set_title(f'start: {start} / end: {end}')

    if save_plot:
        
        plt.savefig(os.path.join(DATA_PATH, 'TOUR_VIZ', '2022', f'{start}_{end}_{tour_cost}.png'))
        
    plt.show()
    
    # CREATE REPORTS
    ###########################################################################
                    
    reports.append(getRouteReport(df_tour, df_distance, checkpoints, mandatory_checkpoints, cp_opening_times, avg_speed=26, reverse=False))
    reports_r.append(getRouteReport(df_tour, df_distance, checkpoints, mandatory_checkpoints, cp_opening_times, avg_speed=26, reverse=True))

# OPTIMAL TOUR ANALYSIS

In [ ]:
tour_costs = [int(x) for x in tour_costs]
plt.bar(range(len(tour_costs)), tour_costs)
plt.ylim(200000, 260000)
plt.show()

In [ ]:
tour_costs_dist = []

for report, report_r in zip(reports, reports_r):
    
    if report.cum_dist_km.iloc[-1] > report_r.cum_dist_km.iloc[-1]:
        
        tour_costs_dist.append(report.cum_dist_km.iloc[-1])
    
    else:
        
        tour_costs_dist.append(report_r.cum_dist_km.iloc[-1])
        

plt.bar(range(len(tour_costs_dist)), [int(x) for x in tour_costs_dist])
plt.ylim(1000, 1350)
plt.show()

In [ ]:
print('best tours (time): ', sorted(range(len(tour_costs)), key=lambda k: tour_costs[k])[:5])
print('best tours (distance): ', sorted(range(len(tour_costs_dist)), key=lambda k: tour_costs_dist[k])[:5])

In [ ]:
reports_r[5]

- SET ALL TRANSPORTS TO ON
- THEN TEST ALL COMBINATIONS OF TRANSPORTS
- THEN SET ALL DIFFERENT DEPARTURE POINTS

- SUBTRACT TIME FOR TRANSPORTS
- RANK BEST ROUTES WRT TOTAL TIME
- ADD TARGET POINTS -> MAYBE WILL HELP FOR GSTEIG IF MORE POINTS IN BE

- compute all optimal routes from all possible start points
- for the shortest routes, compute all possible pairs of transports -> then check eta and feasability
- for the shortest routes, compute eta at each stop -> see if transport is feasible

- OTHER APPROACH: encore transport in distance matrix

- ELEVATION API ? 
- REQUIRE HALF MATRIX (SYMMETRY)
- COMPUTE DIFF(ALT1, ALT2) BETWEEN 2 POINTS
- ESTIMATE SPEEDS FOR ALL GRADIENTS -> DYNAMIC SPEED COMPUTATION